### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv("example.env")
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL 
query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
            + f'&fq={filter_query}&sort={sort}&fl={field_list}')

response = requests.get(query_url).json()
review_list = response["response"]["docs"]
print(json.dumps(review_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
for page in range(0,19):
    # create query with a page number
    page_query_url = f"{query_url}&page={page}"
    
    # API results show 10 articles at a time
    
    
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    reviews.append(response)
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews.append(review)
            # Print the page that was just retrieved
            print(f"Retrieved review for page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No reviews found for page {page}")
        
    page += 1

        


Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 0
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 1
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 2
Retrieved review for page 3
Retrieved review for page 3
Retrieved review for page 3
Retrieved review for page 3
Retrieved review for page 3
Retrieved review for

In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews[:5], indent=4))

[
    {
        "status": "OK",
        "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
        "response": {
            "docs": [
                {
                    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                    "source": "The New York Times",
                    "headline": {
                        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                        "kicker": null,
                        "content_kicker": null,
                        "print_headline": "The Attachment Diaries",
                        "name": null,
                        "seo": null,
                        "sub": null
                    },
                    "keywords": [
                        {
                            "na

In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews)
df = pd.DataFrame(df)
df.head()

,status,copyright,response.docs,response.meta.hits,response.meta.offset,response.meta.time,web_url,snippet,source,keywords,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,OK,Copyright (c) 2024 The New York Times Company....,[{'web_url': 'https://www.nytimes.com/2023/05/...,344.0,0.0,22.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",...,"‘The Attachment Diaries’ Review: Love, Sick",None,NaN,The Attachment Diaries,NaN,NaN,NaN,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,NaN,What’s Love Got to Do With It?,NaN,NaN,NaN,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",...,‘You Can Live Forever’ Review: Do You Love Me ...,None,NaN,You Can Live Forever,NaN,NaN,NaN,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,NaN,A Tourist’s Guide to Love,NaN,NaN,NaN,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",NaN


In [61]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
df["title"] = df["headline.main"]
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df["title"] = df["title"].str.replace('\u2018', '').str.replace('\u2019', '').apply(lambda x: x.strip() + " Review")

/var/folders/j8/8fz1pphj3d18kslnjh7scn_00000gn/T/ipykernel_14886/1126733496.py:7: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df["title"] = df["title"].str.strip(" ' ").replace(" ")


In [62]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string


In [63]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database


### Access The Movie Database API

In [64]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
query_url = url + tmdb_key_string
response = requests.get(query_url).json()
response

{'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}

In [65]:
# Create an empty list to store the results
movie_results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
total_request = 0


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



In [66]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [67]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [68]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [69]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [70]:
# Drop "byline.person" column


In [71]:
# Delete duplicate rows and reset index


In [72]:
# Export data to CSV without the index
